In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv("./lesion-csv.csv")

In [3]:
df.head()

,Unnamed: 0,imageId,teethNumbers,description,numberOfCanals,date,sequenceNumber,lesion
0,0,1-lesion.JPG,17pulpnecrosis,PTNGP,3c,2018-01-09,f4,1
1,1,2-lesion.JPG,121121,"exam,AAP,ChrAbscess,ReTx",NaN,2016-07-05,f1,1
2,2,3-lesion.JPG,1211,"exam,CAP",NaN,2011-03-09,f2,1
3,3,4-lesion.JPG,47,"AAP,WOGGP",3c,2018-06-19,f2,1
4,4,5-lesion.JPG,41313233,Surgery,NaN,2018-06-12,f4,1


In [4]:
df_baseline = df.drop(['Unnamed: 0', 'teethNumbers', 'description', 'numberOfCanals', 'date', 'sequenceNumber'], axis=1)
df_baseline.head()

,imageId,lesion
0,1-lesion.JPG,1
1,2-lesion.JPG,1
2,3-lesion.JPG,1
3,4-lesion.JPG,1
4,5-lesion.JPG,1


In [5]:
print("Total examples: " + str(df_baseline['lesion'].count()))
print("Total lesion images: " + str(df_baseline['lesion'].sum()))
print("Total no lesion images: " + str(df_baseline['lesion'].count() - df_baseline['lesion'].sum()))

Total examples: 399
Total lesion images: 187
Total no lesion images: 212


In [21]:
from imageio import imread

In [23]:
import os

# What are some of the image dimensions?
image_dims = []
bad_files = []

image_dir = "lesion-images-with-id"
image_filenames = os.listdir(image_dir)

for image in image_filenames:
    if image.endswith(".JPG"):
        try:
            image_path = os.path.join(image_dir, image)
            img = Image.open(image_path)
            img.verify()
            image_dim = imread(image_path).shape
            if (image_dim not in image_dims):
                image_dims.append(image_dim)
        except (IOError, SyntaxError) as e:
            bad_files.append(image)

image_dims

[(1708, 2400, 3), (2400, 1708, 3), (803, 620, 3), (900, 640, 3), (805, 620, 3)]

In [24]:
# Looks like some files got corrupted during the data processing
# Back to the drawing board
bad_files

['75-lesion.JPG',
 '302-nolesion.JPG',
 '88-lesion.JPG',
 '384-nolesion.JPG',
 '77-lesion.JPG',
 '390-nolesion.JPG',
 '157-lesion.JPG',
 '204-nolesion.JPG',
 '343-nolesion.JPG',
 '234-nolesion.JPG',
 '85-lesion.JPG',
 '270-nolesion.JPG',
 '323-nolesion.JPG',
 '357-nolesion.JPG',
 '328-nolesion.JPG',
 '243-nolesion.JPG',
 '345-nolesion.JPG',
 '117-lesion.JPG',
 '356-nolesion.JPG',
 '293-nolesion.JPG',
 '226-nolesion.JPG',
 '164-lesion.JPG',
 '155-lesion.JPG']

In [26]:
print("Image Ratios:")
print(image_dims[0][1] / image_dims[0][0])
print(image_dims[1][0] / image_dims[1][1])
print(image_dims[2][0] / image_dims[2][1])
print(image_dims[3][0] / image_dims[3][1])
print(image_dims[4][0] / image_dims[4][1])

# Base on these ratios, to normalize the images, first we should resize, then we should crop

Image Ratios:
1.405152224824356
1.405152224824356
1.2951612903225806
1.40625
1.2983870967741935


In [ ]:
from PIL import Image

# Max size before crop
max_height = 620

for image in image_filenames:
    if image.endswith(".JPG"):
        try:
            image_path = os.path.join(image_dir, image)
            outfile = os.path.splitext(image)[0] + '.resized.JPG'
            
            img = Image.open(image_path)
            img.load()
            
            image_dim = imread(image_path).shape
            if image_dim == (1708, 2400, 3):
                # rotate image.
                img.rotate(90).show()
            
            hpercent = (max_height/float(img.size[1]))
            wsize = int((float(img.size[0])*float(hpercent)))
            #img.resize((wsize,max_height), Image.ANTIALIAS)
            #img.save(outfile, "JPEG")
            
        except (IOError, SyntaxError) as e:
            # Corrupted Image
            continue